PROCESSING GHANA'S INFLATION DATA

In [54]:
#Import Libraries
import pandas as pd
import string
import numpy as np
import random 
from random import shuffle
import re
import nltk
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
import os

In [55]:
tweets = pd.read_csv("inflationGhana.csv")

In [56]:
tweets.head(5)

,Unnamed: 0,User_ID,Name,Username,Tweet_Date,Tweet_language,Tweet_text,Bio
0,0,1718863266,PEDRI+ARAÚJO=🎯,BoatengDuodu,2022-09-23 22:51:57+00:00,en,RT @steve_hanke: Leading member of the New Pat...,football ⚽️
1,1,1336515210,Gbevu♥️,_Oziiiiii,2022-09-23 22:51:14+00:00,en,RT @steve_hanke: Leading member of the New Pat...,…
2,2,723658136,Joshua M Davies,DaviessoJoshua,2022-09-23 22:40:44+00:00,en,RT @steve_hanke: Leading member of the New Pat...,@Acmilan ❤️ @Pluzzfm89.9/ PA/Supervisor/Thalje...
3,3,563636648,Paapa #MTTU,Kandialla_,2022-09-23 22:39:21+00:00,en,RT @steve_hanke: Leading member of the New Pat...,@ManUtd|| @HeartsOfOakGH|| #ManiFAN|| \nKullu ...
4,4,707159736,ace,_Akwasii__,2022-09-23 22:34:58+00:00,en,RT @steve_hanke: Leading member of the New Pat...,Living


In [57]:
tweets.drop('Unnamed: 0', axis=1, inplace=True)

In [58]:
tweets.shape

(1664, 7)

In [59]:
for col in tweets.columns:
    print(col, tweets[col].isnull().sum())

User_ID 0
Name 0
Username 0
Tweet_Date 0
Tweet_language 0
Tweet_text 0
Bio 137


In [60]:

tweets = tweets.loc[:, ['Tweet_text']] 
tweets.head()

,Tweet_text
0,RT @steve_hanke: Leading member of the New Pat...
1,RT @steve_hanke: Leading member of the New Pat...
2,RT @steve_hanke: Leading member of the New Pat...
3,RT @steve_hanke: Leading member of the New Pat...
4,RT @steve_hanke: Leading member of the New Pat...


In [61]:
tweets.drop_duplicates(inplace=True)

In [62]:
#cleaning Text (RT,Punctuation,etc)
tweets_list                 = pd.DataFrame(tweets)
tweets_list["tweetsText"]   = tweets_list["Tweet_text"]

#Removing RT, Punctuation etc
remove_rt                   = lambda x: re.sub('RT @\w+: '," ",x)
rt                          = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tweets_list["tweetsText"]   = tweets_list.tweetsText.map(remove_rt).map(rt)
tweets_list.head(5)

,Tweet_text,tweetsText
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...


In [64]:
tweets['tokenized'] = tweets['tweetsText'].apply(word_tokenize)
tweets.head()

,Tweet_text,tweetsText,tokenized
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,..."


In [65]:
tweets['lower'] = tweets['tokenized'].apply(lambda x: [word.lower() for word in x])
tweets.head()

,Tweet_text,tweetsText,tokenized,lower
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par...","[leading, member, of, the, new, patriotic, par..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi...","[the, chief, executive, officer, of, dalex, fi..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,..."


In [66]:
punc = string.punctuation
tweets['no_punc'] = tweets['lower'].apply(lambda x: [word for word in x if word not in punc])
tweets.head()

,Tweet_text,tweetsText,tokenized,lower,no_punc
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par...","[leading, member, of, the, new, patriotic, par...","[leading, member, of, the, new, patriotic, par..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi...","[the, chief, executive, officer, of, dalex, fi...","[the, chief, executive, officer, of, dalex, fi..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,..."


In [67]:
stop_words = set(stopwords.words('english'))
tweets['stopwords_removed'] = tweets['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
tweets.head()

,Tweet_text,tweetsText,tokenized,lower,no_punc,stopwords_removed
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par...","[leading, member, of, the, new, patriotic, par...","[leading, member, of, the, new, patriotic, par...","[leading, member, new, patriotic, party, npp, ..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, economic, woes, continue, mount, today..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, countries, likely, default..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi...","[the, chief, executive, officer, of, dalex, fi...","[the, chief, executive, officer, of, dalex, fi...","[chief, executive, officer, dalex, finance, ke..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, measuring, ghana, inflation]"


In [68]:
tweets['pos_tags'] = tweets['stopwords_removed'].apply(nltk.tag.pos_tag)
tweets.head()

,Tweet_text,tweetsText,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par...","[leading, member, of, the, new, patriotic, par...","[leading, member, of, the, new, patriotic, par...","[leading, member, new, patriotic, party, npp, ...","[(leading, VBG), (member, NN), (new, JJ), (pat..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, economic, woes, continue, mount, today...","[(ghana, JJ), (economic, JJ), (woes, NNS), (co..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, countries, likely, default...","[(ghana, JJ), (ranks, VBZ), (2nd, CD), (countr..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi...","[the, chief, executive, officer, of, dalex, fi...","[the, chief, executive, officer, of, dalex, fi...","[chief, executive, officer, dalex, finance, ke...","[(chief, JJ), (executive, NN), (officer, NN), ..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, measuring, ghana, inflation]","[(hanke, NN), (everyday, JJ), (u, JJ), (measur..."


In [69]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [70]:
tweets['wordnet_pos'] = tweets['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
tweets.head()

,Tweet_text,tweetsText,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par...","[leading, member, of, the, new, patriotic, par...","[leading, member, of, the, new, patriotic, par...","[leading, member, new, patriotic, party, npp, ...","[(leading, VBG), (member, NN), (new, JJ), (pat...","[(leading, v), (member, n), (new, a), (patriot..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, economic, woes, continue, mount, today...","[(ghana, JJ), (economic, JJ), (woes, NNS), (co...","[(ghana, a), (economic, a), (woes, n), (contin..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, countries, likely, default...","[(ghana, JJ), (ranks, VBZ), (2nd, CD), (countr...","[(ghana, a), (ranks, v), (2nd, n), (countries,..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi...","[the, chief, executive, officer, of, dalex, fi...","[the, chief, executive, officer, of, dalex, fi...","[chief, executive, officer, dalex, finance, ke...","[(chief, JJ), (executive, NN), (officer, NN), ...","[(chief, a), (executive, n), (officer, n), (da..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, measuring, ghana, inflation]","[(hanke, NN), (everyday, JJ), (u, JJ), (measur...","[(hanke, n), (everyday, a), (u, a), (measuring..."


In [71]:
wnl = WordNetLemmatizer()
tweets['lemmatized'] = tweets['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
tweets.head()

,Tweet_text,tweetsText,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,RT @steve_hanke: Leading member of the New Pat...,Leading member of the New Patriotic Party NP...,"[Leading, member, of, the, New, Patriotic, Par...","[leading, member, of, the, new, patriotic, par...","[leading, member, of, the, new, patriotic, par...","[leading, member, new, patriotic, party, npp, ...","[(leading, VBG), (member, NN), (new, JJ), (pat...","[(leading, v), (member, n), (new, a), (patriot...","[lead, member, new, patriotic, party, npp, gab..."
11,RT @steve_hanke: #Ghana's economic woes contin...,Ghana s economic woes continue to mount Tod...,"[Ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, s, economic, woes, continue, to, mount...","[ghana, economic, woes, continue, mount, today...","[(ghana, JJ), (economic, JJ), (woes, NNS), (co...","[(ghana, a), (economic, a), (woes, n), (contin...","[ghana, economic, woe, continue, mount, today,..."
16,RT @steve_hanke: #Ghana ranks 2nd in the count...,Ghana ranks 2nd in the countries most likely...,"[Ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, in, the, countries, most, ...","[ghana, ranks, 2nd, countries, likely, default...","[(ghana, JJ), (ranks, VBZ), (2nd, CD), (countr...","[(ghana, a), (ranks, v), (2nd, n), (countries,...","[ghana, rank, 2nd, country, likely, default, d..."
20,RT @NsiaYaw2: The Chief Executive Officer of D...,The Chief Executive Officer of Dalex Finance ...,"[The, Chief, Executive, Officer, of, Dalex, Fi...","[the, chief, executive, officer, of, dalex, fi...","[the, chief, executive, officer, of, dalex, fi...","[chief, executive, officer, dalex, finance, ke...","[(chief, JJ), (executive, NN), (officer, NN), ...","[(chief, a), (executive, n), (officer, n), (da...","[chief, executive, officer, dalex, finance, ke..."
25,@steve_hanke Everyday u are measuring Ghana's ...,hanke Everyday u are measuring Ghana s infla...,"[hanke, Everyday, u, are, measuring, Ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, are, measuring, ghana, s,...","[hanke, everyday, u, measuring, ghana, inflation]","[(hanke, NN), (everyday, JJ), (u, JJ), (measur...","[(hanke, n), (everyday, a), (u, a), (measuring...","[hanke, everyday, u, measure, ghana, inflation]"


In [72]:
with open('inflation.pkl', 'wb') as pickle_file:
    pickle.dump(tweets, pickle_file)